<a href="https://colab.research.google.com/github/VitaliiSviatoshenko/duckdb/blob/main/%D0%91%D0%B0%D0%B7%D0%BE%D0%B2%D1%96_%D1%84%D1%83%D0%BD%D0%BA%D1%86%D1%96%D1%97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# Імпортуємо бібліотеки pandas для роботи з таблицями
# та sqlite3 для створення тимчасової бази даних SQLite у пам’яті
import pandas as pd
import sqlite3


In [13]:
# Завантажуємо файл з локального комп’ютера через інтерфейс Google Colab
# З'явиться кнопка, де потрібно вибрати CSV-файл
from google.colab import files
uploaded = files.upload()


Saving facebook_ads_basic_daily_202505191056.csv to facebook_ads_basic_daily_202505191056 (1).csv


In [14]:
df = pd.read_csv("facebook_ads_basic_daily_202505191056.csv")
df.head()  # переглянемо перші рядки


,ad_date,campaign_id,adset_id,spend,impressions,reach,clicks,leads,value,url_parameters,total
0,2021-11-30,+1TVQ0Ksk5C1R7k9dKSrpg==,kjdNzjgMLmHu9QXPlKaQHQ==,2686.0,5252.0,4624.0,13.0,NaN,4058.0,utm_source=facebook&utm_medium=ppc&utm_campaig...,NaN
1,2022-04-12,pf4uuqaAAFuHa7gEJnQVKg==,+/ujgC+9ZiLznb2LxP6ihw==,25154.0,4578.0,16804.0,298.0,NaN,31921.0,utm_source=facebook&utm_medium=ppc&utm_campaig...,NaN
2,2021-06-17,+1TVQ0Ksk5C1R7k9dKSrpg==,QBLf/5REFjL80pmdlrLGCw==,4242.0,6045.0,2323.0,97.0,0.0,5597.0,NaN,NaN
3,2022-01-31,h4hWTQf6icvBNjHfA+zKhw==,MVc+HmWqkmrlUI6Iyjdw4w==,1693.0,19554.0,3318.0,80.0,2.0,3383.0,utm_source=facebook&utm_medium=ppc&utm_campaig...,NaN
4,2022-04-05,pf4uuqaAAFuHa7gEJnQVKg==,+/ujgC+9ZiLznb2LxP6ihw==,45152.0,19189.0,12587.0,244.0,7.0,53175.0,utm_source=facebook&utm_medium=ppc&utm_campaig...,NaN


In [6]:
conn = sqlite3.connect(":memory:")  # база в оперативній пам’яті
df.to_sql("ads_data", conn, index=False, if_exists="replace")  # завантажуємо дані в таблицю


2467

In [15]:
# Виконуємо SQL-запит для обчислення ключових метрик по кожному campaign_id і даті:
# - загальні витрати (spend), покази (impressions), кліки (clicks), цінність (value)
# - CPC (вартість кліка), CPM (вартість за 1000 показів), CTR (клікабельність), ROMI (окупність інвестицій)
query = """
SELECT
  ad_date,
  campaign_id,
  SUM(spend) AS total_spend,
  SUM(impressions) AS total_impressions,
  SUM(clicks) AS total_clicks,
  SUM(value) AS total_value,
  CASE WHEN SUM(clicks) > 0 THEN SUM(spend) * 1.0 / SUM(clicks) ELSE 0 END AS cpc,
  CASE WHEN SUM(impressions) > 0 THEN 1000 * SUM(spend) * 1.0 / SUM(impressions) ELSE 0 END AS cpm,
  CASE WHEN SUM(impressions) > 0 THEN SUM(clicks) * 1.0 / SUM(impressions) ELSE 0 END AS ctr,
  CASE WHEN SUM(spend) > 0 THEN (SUM(value) - SUM(spend)) * 1.0 / SUM(spend) ELSE 0 END AS romi
FROM ads_data
WHERE clicks > 0 AND impressions > 0 AND spend > 0
GROUP BY ad_date, campaign_id
ORDER BY ad_date DESC;
"""

result = pd.read_sql_query(query, conn)
result


,ad_date,campaign_id,total_spend,total_impressions,total_clicks,total_value,cpc,cpm,ctr,romi
0,2022-10-31,pf4uuqaAAFuHa7gEJnQVKg==,22745.0,6054.0,58.0,19187.0,392.155172,3757.020152,0.009580,-0.156430
1,2022-10-30,pf4uuqaAAFuHa7gEJnQVKg==,33591.0,27562.0,69.0,47261.0,486.826087,1218.743197,0.002503,0.406954
2,2022-10-29,pf4uuqaAAFuHa7gEJnQVKg==,71403.0,33358.0,115.0,68034.0,620.895652,2140.506026,0.003447,-0.047183
3,2022-10-28,pf4uuqaAAFuHa7gEJnQVKg==,63033.0,5522.0,121.0,59000.0,520.933884,11414.885911,0.021912,-0.063982
4,2022-10-27,pf4uuqaAAFuHa7gEJnQVKg==,35963.0,45223.0,87.0,37669.0,413.367816,795.236937,0.001924,0.047438
...,...,...,...,...,...,...,...,...,...,...
1438,2020-11-15,OL3wfqCd6TPOleVAcp2yew==,698.0,12465.0,657.0,1096.0,1.062405,55.996791,0.052708,0.570201
1439,2020-11-14,OL3wfqCd6TPOleVAcp2yew==,727.0,5005.0,392.0,1267.0,1.854592,145.254745,0.078322,0.742779
1440,2020-11-13,OL3wfqCd6TPOleVAcp2yew==,622.0,11603.0,451.0,960.0,1.379157,53.606826,0.038869,0.543408
1441,2020-11-12,OL3wfqCd6TPOleVAcp2yew==,2300.0,10473.0,397.0,2113.0,5.793451,219.612336,0.037907,-0.081304


In [16]:
# Додатковий SQL-запит:
# - агрегація по campaign_id
# - розрахунок ROMI
# - фільтрація лише тих кампаній, у яких витрати перевищують 500 000
query = """
SELECT
  campaign_id,
  SUM(spend) AS total_spend,
  SUM(value) AS total_value,
  CASE
    WHEN SUM(spend) > 0 THEN (SUM(value) - SUM(spend)) * 1.0 / SUM(spend)
    ELSE 0
  END AS romi
FROM ads_data
GROUP BY campaign_id
HAVING SUM(spend) > 500000;
"""

result = pd.read_sql_query(query, conn)
result


,campaign_id,total_spend,total_value,romi
0,+1TVQ0Ksk5C1R7k9dKSrpg==,1418171.0,1742133.0,0.228436
1,FgcthDK8dH5eIxOj5go3CA==,6363109.0,8023470.0,0.260935
2,Kek1yMtTa5BOTQQs2/uEPg==,2392042.0,2916938.0,0.219434
3,pf4uuqaAAFuHa7gEJnQVKg==,6721282.0,8328866.0,0.239178
4,wHoozjerL5GYr6gvvxnn1Q==,1132697.0,1397463.0,0.233748
